# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [89]:
import pandas as pd
import numpy as np

df = pd.read_csv('tic-tac-toe.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [70]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the data
for column in ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']:
    df[column] = label_encoder.fit_transform(df[column])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TL      958 non-null    int32
 1   TM      958 non-null    int32
 2   TR      958 non-null    int32
 3   ML      958 non-null    int32
 4   MM      958 non-null    int32
 5   MR      958 non-null    int32
 6   BL      958 non-null    int32
 7   BM      958 non-null    int32
 8   BR      958 non-null    int32
 9   class   958 non-null    int64
dtypes: int32(9), int64(1)
memory usage: 41.3 KB


In [71]:
df_input = df.iloc[:,:9]
df_output = df.iloc[:,9:10]
df_output.head()

,class
0,1
1,1
2,1
3,1
4,1


In [72]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_input), columns=df_input.columns)
df_normalized.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1.0,1.0,1.0,1.0,0.5,0.5,1.0,0.5,0.5
1,1.0,1.0,1.0,1.0,0.5,0.5,0.5,1.0,0.5
2,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,1.0
3,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.0,0.0
4,1.0,1.0,1.0,1.0,0.5,0.5,0.0,0.5,0.0


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_normalized,df_output, random_state=66, test_size=.2)

In [103]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, activation= 'relu', input_shape= (9,)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,666 (37.76 KB)

 Trainable params: 9,666 (37.76 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
from keras.optimizers import Adam

my_opt = Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = my_opt,  
              metrics=['accuracy'])

In [98]:
batch_size = 15
epochs = 150

tic_tac_toe = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0119 - val_accuracy: 0.9740 - val_loss: 0.0706
Epoch 2/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9960 - loss: 0.0109 - val_accuracy: 0.9844 - val_loss: 0.0454
Epoch 3/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9959 - loss: 0.0104 - val_accuracy: 0.9740 - val_loss: 0.0684
Epoch 4/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9931 - loss: 0.0226 - val_accuracy: 0.9792 - val_loss: 0.0504
Epoch 5/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9941 - loss: 0.0205 - val_accuracy: 0.9635 - val_loss: 0.1561
Epoch 6/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9927 - loss: 0.0190 - val_accuracy: 0.9792 - val_loss: 0.0643
Epoch 7/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9968 - loss: 0.0117 - val_accuracy: 0.9740 - val_loss: 0.0626
Epoch 8/150
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9966 - loss: 0.0140 - val_accuracy: 0.9479 - v

In [80]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07460670173168182
Test accuracy: 0.9791666865348816


In [106]:
from keras.models import load_model

model.save("tic_tac_toe.h5")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [110]:
import random

# Load the model
tic_tac = load_model('tic_tac_toe.h5')

# Generate 4 random numbers between 0 and 957
random_numbers = [random.randint(0, 957) for _ in range(4)]

for i in random_numbers:
    # Get a random row from the dataset to make predictions
    test_data = df_normalized.iloc[i].to_numpy().reshape(-1,9)

    # Get the correct output value of that row to compare the result
    correct_prediction = int(df_output.iloc[i].values)

    # Use the model to make the prediction
    pred1 = [x for z in (model.predict(test_data).tolist()) for x in z]

    # Compare the prediction to the correct value and print the result
    if pred1[1] > 0.5:
        if 1 == correct_prediction:
            print("The model predicted succesfully.")
            print("Player X won the game")
        else:
            print("The model did not predict successfully")
    else:
        if 0 == correct_prediction:
            print("The model predicted succesfully.")
            print("Player o won the game or the game ended in a draw")
        else:
            print("The model did not predict successfully")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
The model did not predict successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
The model did not predict successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

C:\Users\pieri\AppData\Local\Temp\ipykernel_13840\2510840336.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  correct_prediction = int(df_output.iloc[i].values)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
The model predicted succesfully.
Player X won the game
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
The model predicted succesfully.
Player X won the game


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here